# Live Demo

## Library imports

### External Libraries

In [ ]:
import pandas as pd
import cv2
import plotly.graph_objects as go
import ruptures as rpt
import numpy as np
import statistics

#Offline plotly
import plotly.offline as pyo
pyo.init_notebook_mode()

### Sweatworks Libraries

In [ ]:
import S3conn # Interaction between S3 bucket and script
from angles_detection import generate_angles_output_with_time # Calculating angles for each frame in video
import get_peaks # Exercise states(Limits positions of a given exercise) identification

## Angles Detection

### Reading video and calculating angles in each frame

In [ ]:
from IPython.display import Image, Video

In [ ]:
#video = "../testVideos/IMG_0358.MOV"
#video = "out of shape Dami.mp4"
video = "trim5882CE12-9588-4F74-9283-966C44E85B92.MOV"

In [ ]:
Video(video)

In [ ]:

Image(filename='pose_tracking_full_body_landmarks.png') 

In [ ]:
angles = pd.DataFrame(generate_angles_output_with_time(cv2.VideoCapture(video)))

In [ ]:
#angles.to_csv("test_angesl.csv", index = False)
angles = pd.read_csv("test_angesl.csv")

#### Output from angles detection

In [ ]:
angles

In [ ]:
# Calculating y axis of the motion wave ( avg(back_angles) + avg(leg_angles) + avg(arm_angles) )
angles["avg_angles"] = get_peaks.calculate_avg_angles(angles)

#### Angles combination output. It's the last column

In [ ]:
angles

In [ ]:
angles = angles.sort_values(by="time").reset_index(drop=True)

In [ ]:
angles

In [ ]:
fig = go.Figure()
fig.add_scatter(x = angles.time/1000, y = angles.avg_angles)

# Ruptures

## Calculating exercise changes for a new video

In [ ]:
algorithm = rpt.Pelt(model="rbf").fit(pd.DataFrame(angles.avg_angles))
result = algorithm.predict(pen=20) 

In [ ]:
angles["breaks"] = 0
for i in result:
    if i < angles.shape[0]:
        angles["breaks"][i] = 1

In [ ]:
fig = go.Figure()
fig.add_scatter(x = angles.index, y = angles.avg_angles)
for i in angles[angles.breaks == 1].index:
    fig.add_vline(i)
fig.show()

## Listing all tested videos already uploaded into S3 (Angles and ruptures)

In [ ]:
#Reading all s3 files ending with "angles.csv"
files = S3conn.find_files("angles.csv")

In [ ]:
#Listing all files found with their vector number
for i in range(len(files)):
    print(i, " ", files[i])

## Plotting different Motion waves

### Loading a csv from previously processed videos

In [ ]:
#Reading angles number
datos = S3conn.get_csv(files[3])
angles = datos

In [ ]:
angles

### Creating and showing chart with motion wave and exercise changes

In [ ]:
fig = go.Figure()
fig.add_scatter(x = (datos["time"]/1000), y = datos["avg_angles"], mode = "markers+lines")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_scatter(x = (datos["time"]/1000), y = datos["avg_angles"], mode = "lines")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_scatter(x = (datos["time"]/1000), y = datos["avg_angles"], mode = "lines")
for i in datos.loc[datos.breaks == 1].index:
    fig.add_vline(x = datos.iloc[i]["time"]/1000)
fig.show()

# Exercise Detection

In [ ]:
import pickle
import tensorflow as tf

In [ ]:
dnn_model = tf.keras.models.load_model("../Training/dnn_model")

In [ ]:
dnn_model.summary()

In [ ]:
et_model = pickle.load(open("et_2.sav", 'rb'))

In [ ]:
chunk = 0
start = angles[angles.breaks ==1].index[chunk]
finish = angles[angles.breaks ==1].index[chunk+1]
angles_to_predict = angles.iloc[range(start, finish)]

In [ ]:
angles_to_predict = angles

In [ ]:
angles_to_predict

In [ ]:
peaks = get_peaks.states(angles_to_predict["avg_angles"])
angles_to_predict["consec"] = range(angles_to_predict.shape[0])
angles_to_predict = angles_to_predict.merge(peaks, on="consec", how="left").fillna(0)

In [ ]:
fig = go.Figure()
fig.add_scatter(x = angles_to_predict.time/1000, y = angles_to_predict.avg_angles, mode = "lines", name = "motion wave")
fig.add_scatter(x = angles_to_predict.loc[angles_to_predict["value"] !=0 ]["time"]/1000, y = angles_to_predict.loc[angles_to_predict["value"] !=0 ]["avg_angles"], mode = "markers", name = "exercise states")
fig.show()

In [ ]:
peaks = angles_to_predict.loc[angles_to_predict.type == "peak"]
peaks.columns = peaks.columns+"_peaks"
valleys = angles_to_predict.loc[angles_to_predict.type == "valley"]
valleys.columns = valleys.columns+"_valleys"
cross_train_data = peaks.merge(right = valleys, how="cross")

In [ ]:
cross_train_data

In [ ]:
#normalización de variables peaks
cross_train_data["elbow_angles_norm_peaks"] = (cross_train_data["elbow_angles_peaks"]/180)
cross_train_data["elbow_angles2_norm_peaks"] = (cross_train_data["elbow_angles2_peaks"]/180)
cross_train_data["armpit_angles_norm_peaks"] = (cross_train_data["armpit_angles_peaks"]/180)
cross_train_data["armpit_angles2_norm_peaks"] = (cross_train_data["armpit_angles2_peaks"]/180)
cross_train_data["legs_angles_norm_peaks"] = (cross_train_data["legs_angles_peaks"]/180)
cross_train_data["legs_angles2_norm_peaks"] = (cross_train_data["legs_angles2_peaks"]/180) 
cross_train_data["back_angles_norm_peaks"] = (cross_train_data["back_angles_peaks"]/180)
cross_train_data["back_angles2_norm_peaks"] = (cross_train_data["back_angles2_peaks"]/180)

#normalización de variables valleys
cross_train_data["elbow_angles_norm_valleys"] = (cross_train_data["elbow_angles_valleys"]/180)
cross_train_data["elbow_angles2_norm_valleys"] = (cross_train_data["elbow_angles2_valleys"]/180)
cross_train_data["armpit_angles_norm_valleys"] = (cross_train_data["armpit_angles_valleys"]/180)
cross_train_data["armpit_angles2_norm_valleys"] = (cross_train_data["armpit_angles2_valleys"]/180)
cross_train_data["legs_angles_norm_valleys"] = (cross_train_data["legs_angles_valleys"]/180)
cross_train_data["legs_angles2_norm_valleys"] = (cross_train_data["legs_angles2_valleys"]/180) 
cross_train_data["back_angles_norm_valleys"] = (cross_train_data["back_angles_valleys"]/180)
cross_train_data["back_angles2_norm_valleys"] = (cross_train_data["back_angles2_valleys"]/180)


In [ ]:
to_predict = cross_train_data.loc[:,["elbow_angles_norm_peaks",
                                                                "elbow_angles_peaks",
                                                                "elbow_angles2_norm_peaks",
                                                                "elbow_angles2_peaks",
                                                                "armpit_angles_norm_peaks",
                                                                "armpit_angles_peaks",
                                                                "armpit_angles2_norm_peaks",
                                                                "armpit_angles2_peaks",
                                                                "legs_angles_norm_peaks",
                                                                "legs_angles_peaks",
                                                                "legs_angles2_norm_peaks",
                                                                "legs_angles2_peaks",
                                                                "back_angles_norm_peaks",
                                                                "back_angles_peaks",
                                                                "back_angles2_norm_peaks",
                                                                "back_angles2_peaks",
                                                                "elbow_angles_norm_valleys",
                                                                "elbow_angles_valleys",
                                                                "elbow_angles2_norm_valleys",
                                                                "elbow_angles2_valleys",
                                                                "armpit_angles_norm_valleys",
                                                                "armpit_angles_valleys",
                                                                "armpit_angles2_norm_valleys",
                                                                "armpit_angles2_valleys",
                                                                "legs_angles_norm_valleys",
                                                                "legs_angles_valleys",
                                                                "legs_angles2_norm_valleys",
                                                                "legs_angles2_valleys",
                                                                "back_angles_norm_valleys",
                                                                "back_angles_valleys",
                                                                "back_angles2_norm_valleys",
                                                                "back_angles2_valleys",
                                                                "slope_peaks",
                                                                "slope_valleys"]]

In [ ]:
predictions = dnn_model.predict(to_predict)

In [ ]:
#import pickle
labels = pickle.load(open("../Training/label_dict.sav", 'rb'))

In [ ]:
inverse_labels = {v: k for k, v in labels.items()}

In [ ]:
class_prediction = []
for i in range(len(predictions)):
    class_prediction.append(inverse_labels[np.argmax(predictions[i])])

In [ ]:
class_prediction

In [ ]:
import statistics

In [ ]:
statistics.mode(class_prediction)

In [ ]:
et_model.predict(to_predict)